# HW3.  Database Design

### Library Relations

- Member = {<span style="text-decoration:underline">email</span>, name, birthday}
- Borrow = {<span style="text-decoration:underline">borrowID</span><sup>FK-BorrowTransactions</sup>, <span style="text-decoration:underline">email</span><sup>FK-Member</sup>,<span style="text-decoration:underline">itemID</span><sup>FK-Item</sup>}
- BorrowTransactions = {<span style="text-decoration:underline">borrowID</span>, borrowDate, returnDate, status}
- Fines = {<span style="text-decoration:underline">borrowID</span><sup>FK-BorrowTransactions</sup>, status, amount}
- Item = {<span style="text-decoration:underline">itemID</span>, name, author, category, genre, status}
- Records = {<span style="text-decoration:underline">itemID</span><sup>FK-Item</sup>, status, lastUpdated}
 -Staff = {<span style="text-decoration:underline">email</span><sup>FK-Member</sup>, <span style="text-decoration:underline">staffID</span>, position, wage, employmentDate}
- Volunteer = {<span style="text-decoration:underline">email</span><sup>FK-Member</sup>, <span style="text-decoration:underline">volunteerID</span>, employmentDate}
- Hold = {{<span style="text-decoration:underline">staffID</span><sup>FK-Staff</sup>>,<span style="text-decoration:underline">eventID</span>}<sup>FK-Events</sup>}
- Events = {<span style="text-decoration:underline">eventID</span>, name, scheduledTime, scheduledDate, targetAudience}
- Located = {<span style="text-decoration:underline">eventID</span><sup>FK-Events</sup>, <span style="text-decoration:underline">roomNum</span><sup>FK-Room</sup>}
- Room = {<span style="text-decoration:underline">roomNum</span>, maxCap}

#### Creating Database

In [5]:
%load_ext sql
%config SqlMagic.displaylimit = None

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


displaylimit: Value None will be treated as 0 (no limit)

In [6]:
%sql sqlite:///library.db

In [7]:
%%sql

CREATE TABLE Member (
    email VARCHAR(500) PRIMARY KEY,
    name VARCHAR(100),
    birthday DATE
);

CREATE TABLE Borrow (
    borrowID INTEGER,
    email INTEGER,
    itemID INTEGER,
    PRIMARY KEY (borrowID, email, itemID),
    FOREIGN KEY (borrowID) REFERENCES BorrowTransactions(borrowID),
    FOREIGN KEY (email) REFERENCES Member(email),
    FOREIGN KEY (itemID) REFERENCES Item(itemID)
);


CREATE TABLE BorrowTransactions (
    borrowID INTEGER PRIMARY KEY,
    borrowDate DATE,
    returnDate DATE DEFAULT NULL,
    status VARCHAR(10) CHECK (status IN ('Borrowed', 'Returned')) DEFAULT 'Borrowed'
);

CREATE TABLE Fines (
    borrowID INTEGER PRIMARY KEY,
    status VARCHAR(6) CHECK (status IN ('Paid', 'Unpaid')),
    amount DECIMAL(10,2) CHECK (amount >= 0),
    FOREIGN KEY (borrowID) REFERENCES BorrowTransactions(borrowID)
    ON DELETE CASCADE
);

CREATE TABLE Item (
    itemID INTEGER PRIMARY KEY,
    name VARCHAR(255),
    author VARCHAR(100),
    category VARCHAR(50),
    genre VARCHAR(50),
    status VARCHAR(11) CHECK (status IN ('Available', 'Unavailable'))
);

CREATE TABLE Records (
    itemID INTEGER,
    status VARCHAR(20) CHECK (status IN ('In System', 'Archived')),
    lastUpdated DATE,
    FOREIGN KEY (itemID) REFERENCES Item(itemID)
);

CREATE TABLE Staff (
    email INTEGER,
    staffID INTEGER PRIMARY KEY,
    position VARCHAR(50),
    wage DECIMAL(10,2),
    employmentDate DATE,
    employmentStatus VARCHAR(20) CHECK (employmentStatus IN ('Working', 'Resigned')),
    FOREIGN KEY (email) REFERENCES Member(email)
);

CREATE TABLE Volunteer (
    email INTEGER,
    volunteerID INTEGER PRIMARY KEY,
    employmentDate DATE,
    FOREIGN KEY (email) REFERENCES Member(email)
);

CREATE TABLE Hold (
    staffID INTEGER,
    eventID INTEGER,
    PRIMARY KEY (staffID, eventID),
    FOREIGN KEY (staffID) REFERENCES Staff(staffID),
    FOREIGN KEY (eventID) REFERENCES Events(eventID)
);

CREATE TABLE Attends(
    email INTEGER,
    eventID,
    PRIMARY KEY (email, eventID),
    FOREIGN KEY (email) REFERENCES Member(email),
    FOREIGN KEY (eventID) REFERENCES Events(eventID)
);

CREATE TABLE Events (
    eventID INTEGER PRIMARY KEY,
    name VARCHAR(100),
    scheduledTime TIME,
    scheduledDate DATE,
    targetAudience VARCHAR(100)
);

CREATE TABLE Located (
    eventID INTEGER,
    roomNum INTEGER,
    PRIMARY KEY (eventID, roomNum),
    FOREIGN KEY (eventID) REFERENCES Events(eventID),
    FOREIGN KEY (roomNum) REFERENCES Room(roomNum)
);

CREATE TABLE Room (
    roomNum INTEGER PRIMARY KEY,
    maxCap INTEGER
);

Running query in 'sqlite:///library.db'

++
||
++
++

#### Triggers

1) Prevent item deletion if it is still borrowed

In [8]:
%%sql
CREATE TRIGGER prevent_item_deletion_if_borrowed
BEFORE DELETE ON Item
FOR EACH ROW
BEGIN
    -- Prevent item deletion if it is still borrowed
    SELECT RAISE(ABORT, 'Item cannot be deleted as it is currently borrowed.')
    WHERE EXISTS (SELECT 1 FROM Borrow WHERE itemID = OLD.itemID);
END;


Running query in 'sqlite:///library.db'

++
||
++
++

2. When add item, also adds record

In [9]:
%%sql

CREATE TRIGGER add_record_on_item_insert
AFTER INSERT ON Item
FOR EACH ROW
BEGIN
    INSERT INTO Records (itemID, status, lastUpdated)
    VALUES (NEW.itemID, 'In System', DATE('now'));
END;

Running query in 'sqlite:///library.db'

++
||
++
++

3. If staff holds event, they also attend the event

In [10]:
%%sql

CREATE TRIGGER staff_attends_event
AFTER INSERT ON Hold
FOR EACH ROW
BEGIN
    INSERT INTO Attends (email, eventID)
    SELECT email, NEW.eventID
    FROM Staff
    WHERE staffID = NEW.staffID;
END;


Running query in 'sqlite:///library.db'

++
||
++
++

4) If an item is overdue in borrows -> Adds to fines

In [11]:
%%sql

CREATE TRIGGER AddFineForOverdueInsert
AFTER INSERT ON BorrowTransactions
FOR EACH ROW
BEGIN
    -- Insert a fine if the status is 'Borrowed' and returnDate is overdue
    INSERT INTO Fines (borrowID, status, amount)
    SELECT New.borrowID, 'Unpaid', 10.00
    WHERE NEW.status = 'Borrowed' AND NEW.returnDate < CURRENT_DATE;
END;



Running query in 'sqlite:///library.db'

++
||
++
++

In [12]:
%%sql

CREATE TRIGGER AddFineForOverdueUpdate
AFTER UPDATE ON BorrowTransactions
FOR EACH ROW
BEGIN
    -- Insert a fine if the status is 'Borrowed' and returnDate is overdue
    INSERT INTO Fines (borrowID, status, amount)
    SELECT New.borrowID, 'Unpaid', 10.00
    WHERE NEW.status = 'Borrowed' AND NEW.returnDate < CURRENT_DATE;
END;


Running query in 'sqlite:///library.db'

++
||
++
++

6) When you insert into borrow transactions -> Makes it return date 1 month after

In [13]:
%%sql

CREATE TRIGGER SetReturnDate
AFTER INSERT ON BorrowTransactions
FOR EACH ROW
BEGIN
    UPDATE BorrowTransactions
    SET returnDate = DATE(NEW.borrowDate, '+1 month')
    WHERE borrowID = NEW.borrowID;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

7) If you are a staff -> You cant be a volunteer

In [14]:
%%sql 
CREATE TRIGGER prevent_volunteer_if_staff
AFTER INSERT ON Volunteer
FOR EACH ROW
BEGIN
    -- Check if the email is already in the Staff table
    DELETE FROM Volunteer
    WHERE email = NEW.email
    AND EXISTS (SELECT 1 FROM Staff WHERE email = NEW.email);
END;

Running query in 'sqlite:///library.db'

++
||
++
++

8) If you wanna be a staff and your a volunteer -> You get deleted from volunteer table

In [15]:
%%sql
CREATE TRIGGER remove_volunteer_if_staff
AFTER INSERT ON Staff
FOR EACH ROW
BEGIN
    -- If email in volunteer table -> Delete
    DELETE FROM Volunteer WHERE email = NEW.email;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

Sets ids so they increase

In [16]:
%%sql
CREATE TRIGGER set_borrow_transactions
BEFORE INSERT ON BorrowTransactions
FOR EACH ROW
WHEN NEW.borrowID IS NULL
BEGIN
    UPDATE BorrowTransactions
    SET borrowID = (
        CASE 
            WHEN (SELECT MAX(borrowID) FROM BorrowTransactions) IS NULL THEN 1
            ELSE (SELECT MAX(borrowID) FROM BorrowTransactions) + 1
        END
    )
    WHERE rowid = NEW.rowid;
END;

CREATE TRIGGER set_staff
BEFORE INSERT ON Staff
FOR EACH ROW
WHEN NEW.staffID IS NULL
BEGIN
    UPDATE Staff
    SET staffID = (
        CASE 
            WHEN (SELECT MAX(staffID) FROM Staff) IS NULL THEN 1
            ELSE (SELECT MAX(staffID) FROM Staff) + 1
        END
    )
    WHERE rowid = NEW.rowid;
END;

CREATE TRIGGER set_volunteer
BEFORE INSERT ON Volunteer
FOR EACH ROW
WHEN NEW.volunteerID IS NULL
BEGIN
    UPDATE Volunteer
    SET volunteerID = (
        CASE 
            WHEN (SELECT MAX(volunteerID) FROM Volunteer) IS NULL THEN 1
            ELSE (SELECT MAX(volunteerID) FROM Volunteer) + 1
        END
    )
    WHERE rowid = NEW.rowid;
END;

CREATE TRIGGER set_event
BEFORE INSERT ON Events
FOR EACH ROW
WHEN NEW.eventID IS NULL
BEGIN
    UPDATE Events
    SET eventID = (
        CASE 
            WHEN (SELECT MAX(eventID) FROM Events) IS NULL THEN 1
            ELSE (SELECT MAX(eventID) FROM Events) + 1
        END
    )
    WHERE rowid = NEW.rowid;
END;

CREATE TRIGGER set_room
BEFORE INSERT ON Room
FOR EACH ROW
WHEN NEW.roomNum IS NULL
BEGIN
    UPDATE Room
    SET roomNum = (
        CASE 
            WHEN (SELECT MAX(roomNum) FROM Room) IS NULL THEN 1
            ELSE (SELECT MAX(roomNum) FROM Room) + 1
        END
    )
    WHERE rowid = NEW.rowid;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [17]:
%%sql
PRAGMA table_info(Fines)

Running query in 'sqlite:///library.db'

cid,name,type,notnull,dflt_value,pk
0,borrowID,INTEGER,0,None,1
1,status,VARCHAR(6),0,None,0
2,amount,"DECIMAL(10,2)",0,None,0


#### Inserts

In [18]:
%%sql
-- Member
-- Member
INSERT INTO Member (email, name, birthday) VALUES
('ash.ketchum@email.com', 'Ash Ketchum', '2006-03-22'),
('pikachu@email.com', 'Pikachu', '2008-07-08'),
('misty@email.com', 'Misty', '2004-08-15'),
('brock@email.com', 'Brock', '2005-11-22'),
('team.rocket@email.com', 'Team Rocket', '2007-02-01'),
('squirtle@email.com', 'Squirtle', '2008-10-15'),
('charmander@email.com', 'Charmander', '2009-04-18'),
('bulbasaur@email.com', 'Bulbasaur', '2008-05-22'),
('emolga@email.com', 'Emolga', '2003-07-23'),
('mimikyu@email.com', 'Mimikyu', '2008-12-31'),
('spongebob@email.com', 'SpongeBob SquarePants', '2004-07-14'),
('patrick@email.com', 'Patrick Star', '2005-02-19'),
('squidward@email.com', 'Squidward Tentacles', '1999-10-09'),
('mr.krabs@email.com', 'Mr. Krabs', '1980-06-30'),
('sandy@email.com', 'Sandy Cheeks', '2005-11-26'),
('plankton@email.com', 'Plankton', '2006-03-13'),
('simba@email.com', 'Simba', '2007-06-15'),
('nala@email.com', 'Nala', '2006-05-20'),
('scar@email.com', 'Scar', '1998-09-30'),
('ariel@email.com', 'Ariel', '2004-11-01'),
('flounder@email.com', 'Flounder', '2005-04-25'),
('ursula@email.com', 'Ursula', '1993-09-06'),
('belle@email.com', 'Belle', '2003-11-22'),
('beast@email.com', 'Beast', '2004-06-17'),
('aladdin@email.com', 'Aladdin', '2005-11-06'),
('genie@email.com', 'Genie', '2004-04-17'),
('sonic@email.com', 'Sonic the Hedgehog', '2003-06-23'),
('tails@email.com', 'Tails', '2004-12-02'),
('knuckles@email.com', 'Knuckles the Echidna', '2003-02-01'),
('robotnik@email.com', 'Dr. Robotnik', '2001-08-25'),
('amyrose@email.com', 'Amy Rose', '2005-05-07'),
('elena@email.com', 'Elena', '2005-04-03'),
('nathan@email.com', 'Nathan', '2004-11-08'),
('luka@email.com', 'Luka', '2021-08-08'),
('mochie@email.com', 'Mochie', '2019-06-20');


Running query in 'sqlite:///library.db'

35 rows affected.

++
||
++
++

In [19]:
%%sql
-- BorrowTransactions
INSERT INTO BorrowTransactions (borrowID, borrowDate, returnDate, status) VALUES
(2000, '2024-01-01', '2024-02-01', 'Returned'),
(NULL, '2025-03-28', '2025-03-29', 'Returned'),
(NULL, '2025-03-27', '2025-03-28', 'Returned'),
(NULL, '2025-03-25', '2025-03-26', 'Returned'),
(NULL, '2024-12-15', '2025-01-15', 'Returned'),
(NULL, '2024-11-20', '2024-12-20', 'Returned'),
(NULL, '2025-03-20', '2025-03-21', 'Returned'),
(NULL, '2025-03-18', '2025-03-19', 'Returned'),
(NULL, '2024-06-01', '2024-07-01', 'Returned'),
(NULL, '2025-03-22', '2025-03-23', 'Returned'),

(NULL, '2024-09-15', NULL, 'Borrowed'),
(NULL, '2024-11-10', NULL, 'Borrowed'),
(NULL, '2025-03-26', NULL, 'Borrowed'),
(NULL, '2024-12-01', NULL, 'Borrowed'),
(NULL, '2024-08-20', NULL, 'Borrowed'),
(NULL, '2024-10-05', NULL, 'Borrowed'),
(NULL, '2024-02-15', NULL, 'Borrowed'), 
(NULL, '2024-11-05', NULL, 'Borrowed'),  
(NULL, '2024-07-12', NULL, 'Borrowed'), 
(NULL, '2024-05-28', NULL, 'Borrowed');  


Running query in 'sqlite:///library.db'

20 rows affected.

++
||
++
++

In [20]:
%%sql
--Items
INSERT INTO Item (itemID, name, author, category, genre, status) VALUES
(4000, 'Harry Potter and the Sorcerer''s Stone', 'J.K. Rowling', 'Book', 'Fantasy', 'Available'),
(NULL, 'Harry Potter and the Chamber of Secrets', 'J.K. Rowling', 'Book', 'Fantasy', 'Available'),
(NULL, 'Harry Potter and the Prisoner of Azkaban', 'J.K. Rowling', 'Book', 'Fantasy', 'Available'),
(NULL, 'The Hobbit', 'J.R.R. Tolkien', 'Book', 'Fantasy', 'Available'),
(NULL, '1984', 'George Orwell', 'Book', 'Dystopian', 'Available'),
(NULL, 'To Kill a Mockingbird', 'Harper Lee', 'Book', 'Classic', 'Available'),
(NULL, 'The Catcher in the Rye', 'J.D. Salinger', 'Book', 'Classic', 'Available'),
(NULL, 'Pride and Prejudice', 'Jane Austen', 'Book', 'Romance', 'Available'),
(NULL, 'Rock Legends Collection', 'Various Artists', 'CD', 'Rock', 'Available'),
(NULL, 'Jazz Classics', 'Various Artists', 'CD', 'Jazz', 'Available'),

(NULL, 'Moby-Dick', 'Herman Melville', 'Book', 'Classic', 'Unavailable'),
(NULL, 'The Great Gatsby', 'F. Scott Fitzgerald', 'Book', 'Classic', 'Unavailable'),
(NULL, 'Brave New World', 'Aldous Huxley', 'Book', 'Dystopian', 'Unavailable'),
(NULL, 'Fahrenheit 451', 'Ray Bradbury', 'Book', 'Dystopian', 'Unavailable'),
(NULL, 'The Lord of the Rings', 'J.R.R. Tolkien', 'Book', 'Fantasy', 'Unavailable'),
(NULL, 'Dracula', 'Bram Stoker', 'Book', 'Horror', 'Unavailable'),
(NULL, 'Beethoven: Symphony No. 9', 'Ludwig van Beethoven', 'CD', 'Classical', 'Unavailable'),
(NULL, 'Pop Hits 2020', 'Various Artists', 'CD', 'Pop', 'Unavailable'),
(NULL, 'The Dark Side of the Moon', 'Pink Floyd', 'CD', 'Rock', 'Unavailable'),
(NULL, 'Hip-Hop Classics', 'Various Artists', 'CD', 'Hip-Hop', 'Unavailable');


Running query in 'sqlite:///library.db'

20 rows affected.

++
||
++
++

when borrowing
- make new borrow transaction
- know: itemid
- know: member id

In [21]:
%%sql

-- Borrow
INSERT INTO Borrow (borrowID, email, itemID) VALUES
(2000, 'ariel@email.com', 4000),
(2001, 'ariel@email.com', 4001),
(2002, 'nathan@email.com', 4002),
(2003, 'elena@email.com', 4003),
(2004, 'team.rocket@email.com', 4004),
(2005, 'elena@email.com', 4005),
(2006, 'luka@email.com', 4006),
(2007, 'luka@email.com', 4007),
(2008, 'emolga@email.com', 4008),
(2009, 'knuckles@email.com', 4009),
(2010, 'pikachu@email.com', 4010),
(2011, 'misty@email.com', 4011),
(2012, 'brock@email.com', 4012),
(2013, 'squirtle@email.com', 4013),
(2014, 'charmander@email.com', 4014),
(2015, 'bulbasaur@email.com', 4015),
(2016, 'mimikyu@email.com', 4016),
(2017, 'spongebob@email.com', 4017),
(2018, 'patrick@email.com', 4018),
(2019, 'squidward@email.com', 4019);


Running query in 'sqlite:///library.db'

20 rows affected.

++
||
++
++

In [22]:
%%sql
--Staff
INSERT INTO Staff (email, staffID, position, wage, employmentDate, employmentStatus) VALUES
('ash.ketchum@email.com', 5000, 'Librarian', 45000.00, '2019-05-15', 'Working'),
('pikachu@email.com', NULL, 'Assistant Librarian', 40000.00, '2020-07-22', 'Working'),
('misty@email.com', NULL, 'Cleaner', 25000.00, '2021-03-30', 'Working'),
('brock@email.com', NULL, 'Security', 28000.00, '2022-09-10', 'Resigned'),
('team.rocket@email.com', NULL, 'Librarian', 46000.00, '2021-01-12', 'Working'),
('squirtle@email.com', NULL, 'Assistant Librarian', 42000.00, '2022-11-05', 'Working'),
('charmander@email.com', NULL, 'Cleaner', 24000.00, '2020-10-28', 'Working'),
('bulbasaur@email.com', NULL, 'Security', 29000.00, '2021-08-19', 'Resigned'),
('emolga@email.com', NULL, 'Librarian', 50000.00, '2018-06-01', 'Working'),
('mimikyu@email.com', NULL, 'Assistant Librarian', 41000.00, '2023-02-15', 'Working');



Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [23]:
%%sql
-- Volunteer
INSERT INTO Volunteer (email, volunteerID, employmentDate) VALUES
('spongebob@email.com', 6000, '2023-01-15'),
('patrick@email.com', NULL, '2023-02-20'),
('squidward@email.com', NULL, '2023-03-25'),
('mr.krabs@email.com', NULL, '2023-04-10'),
('sandy@email.com', NULL, '2023-05-05'),
('plankton@email.com', NULL, '2023-06-18'),
('simba@email.com', NULL, '2023-07-22'),
('nala@email.com', NULL, '2023-08-14'),
('scar@email.com', NULL, '2023-09-09'),
('ariel@email.com', NULL, '2023-10-02');


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [24]:
%%sql
-- Event
INSERT INTO Events (eventID, name, scheduledTime, scheduledDate, targetAudience) VALUES
(7000, 'Library Book Reading', '14:00:00', '2025-05-15', 'All Ages'),
(NULL, 'Children’s Storytime', '10:30:00', '2025-05-18', 'Children'),
(NULL, 'Coding Workshop', '16:00:00', '2025-05-20', 'Teens and Adults'),
(NULL, 'Author Meet and Greet', '12:00:00', '2025-03-22', 'Adults'),
(NULL, 'Community Clean-Up', '08:00:00', '2025-02-04', 'Volunteers'),
(NULL, 'Book Donation Drive', '09:00:00', '2025-02-04', 'All Ages'),
(NULL, 'Film Screening', '18:00:00', '2025-02-03', 'Teens and Adults'),
(NULL, 'Public Lecture on History', '19:00:00', '2025-01-05', 'Adults'),
(NULL, 'Holiday Craft Workshop', '14:00:00', '2024-12-10', 'Children and Families'),
(NULL, 'New Year’s Eve Party', '20:00:00', '2024-12-31', 'All Ages');


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [25]:
%%sql

--Attends
INSERT INTO Attends (email, eventID) VALUES
('pikachu@email.com', 7000),
('misty@email.com', 7001),
('brock@email.com', 7002),
('team.rocket@email.com', 7002),
('squirtle@email.com', 7003),
('charmander@email.com', 7004),
('bulbasaur@email.com', 7005),
('emolga@email.com', 7006),
('mimikyu@email.com', 7007),
('spongebob@email.com', 7008),
('patrick@email.com', 7000),
('squidward@email.com', 7001),
('mr.krabs@email.com', 7002),
('sandy@email.com', 7003),
('plankton@email.com', 7004),
('simba@email.com', 7005),
('nala@email.com', 7006),
('scar@email.com', 7007),
('ariel@email.com', 7008),
('flounder@email.com', 7009),
('ursula@email.com', 7000),
('belle@email.com', 7001),
('beast@email.com', 7002),
('aladdin@email.com', 7003),
('genie@email.com', 7004),
('sonic@email.com', 7005),
('tails@email.com', 7006),
('knuckles@email.com', 7007),
('robotnik@email.com', 7008),
('amyrose@email.com', 7009),
('elena@email.com', 7000),
('nathan@email.com', 7001),
('luka@email.com', 7002),
('mochie@email.com', 7003);



Running query in 'sqlite:///library.db'

34 rows affected.

++
||
++
++

In [26]:
%%sql
--Holds
INSERT INTO Hold (staffID, eventID) VALUES
(5000, 7000),
(5001, 7001),
(5002, 7002),
(5003, 7003),
(5004, 7004),
(5005, 7005),
(5006, 7006),
(5007, 7007),
(5008, 7008),
(5009, 7009);

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [27]:
%%sql
-- Room
INSERT INTO Room (roomNum, maxCap) VALUES
(8000, 20),
(NULL, 30),
(NULL, 20),
(NULL, 5),
(NULL, 10),
(NULL, 40),
(NULL, 60),
(NULL, 20),
(NULL, 30),
(NULL, 25);

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [28]:
%%sql
--Located
INSERT INTO Located (eventID, roomNum) VALUES
(7000, 8000),
(7001, 8001),
(7002, 8002),
(7003, 8003),
(7004, 8004),
(7005, 8005),
(7006, 8006),
(7007, 8007),
(7008, 8008),
(7009, 8009);


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [33]:
%%sql
SELECT * FROM Attends;

Running query in 'sqlite:///library.db'

email,eventID
pikachu@email.com,7000
misty@email.com,7001
brock@email.com,7002
team.rocket@email.com,7002
squirtle@email.com,7003
charmander@email.com,7004
bulbasaur@email.com,7005
emolga@email.com,7006
mimikyu@email.com,7007
spongebob@email.com,7008


#### Indexes

In [30]:
%%sql

CREATE INDEX ItemIndex ON Item(name, author);

Running query in 'sqlite:///library.db'

++
||
++
++

#### FOR TESTING PURPOSES
Delete tables if you need to

In [31]:
# %%sql 
# -- Disable foreign key checks temporarily
# PRAGMA foreign_keys = OFF;

# -- Drop all tables (starting with those with foreign keys)
# DROP TABLE IF EXISTS Hold;
# DROP TABLE IF EXISTS Located;
# DROP TABLE IF EXISTS Attends;
# DROP TABLE IF EXISTS Events;
# DROP TABLE IF EXISTS Staff;
# DROP TABLE IF EXISTS Volunteer;
# DROP TABLE IF EXISTS Borrow;
# DROP TABLE IF EXISTS BorrowTransactions;
# DROP TABLE IF EXISTS Records;
# DROP TABLE IF EXISTS Fines;
# DROP TABLE IF EXISTS Owes;
# DROP TABLE IF EXISTS Item;
# DROP TABLE IF EXISTS Member;
# DROP TABLE IF EXISTS Room;

# -- Re-enable foreign key checks
# PRAGMA foreign_keys = ON;
